In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

print("Libraries Imported!")

Libraries Imported!


In [52]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")


In [53]:
train_data.head(20)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.00,False,0.00,0.00,0.00,0.00,0.00,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.00,False,109.00,9.00,25.00,549.00,44.00,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.00,True,43.00,3576.00,0.00,6715.00,49.00,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.00,False,0.00,1283.00,371.00,3329.00,193.00,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.00,False,303.00,70.00,151.00,565.00,2.00,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.00,False,0.00,483.00,0.00,291.00,0.00,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.00,False,42.00,1539.00,3.00,0.00,0.00,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.00,False,0.00,0.00,0.00,0.00,nan,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.00,False,0.00,785.00,17.00,216.00,0.00,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.00,False,0.00,0.00,0.00,0.00,0.00,Erraiam Flatic,True


In [54]:
train_data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [55]:
'''plt.figure(figsize=(20,20))
plt.subplot(1,5,1)
sb.countplot(data=train_data,x="CryoSleep" )
plt.subplot(1,5,2)
sb.countplot(data=train_data,x="Destination" )
plt.subplot(1,5,3)
sb.countplot(data=train_data[train_data['Side'] == 'P'],x="Transported" )
plt.subplot(1,5,4)
sb.countplot(data=train_data,x="Deck" )
'''

'plt.figure(figsize=(20,20))\nplt.subplot(1,5,1)\nsb.countplot(data=train_data,x="CryoSleep" )\nplt.subplot(1,5,2)\nsb.countplot(data=train_data,x="Destination" )\nplt.subplot(1,5,3)\nsb.countplot(data=train_data[train_data[\'Side\'] == \'P\'],x="Transported" )\nplt.subplot(1,5,4)\nsb.countplot(data=train_data,x="Deck" )\n'

In [56]:
def data_to_int(df):
    try:
        df['Transported'] = df['Transported'].astype(int)
    except:
        pass
    df['CryoSleep'] = df['CryoSleep'].astype(int)
    df['VIP'] = df['VIP'].astype(int)

    deck = []
    #num = []
    side = []
    for cabin in df['Cabin']:
        #print(cabin)
        deck.append(cabin.split('/')[0])
        #num.append(cabin.split('/')[1])
        side.append(cabin.split('/')[2])
    #print(deck)
    print(f"len deck = {len(deck)}")
    print(f"len side = {len(side)}")
    df = df.drop(['Cabin'],axis=1)
    df.insert(2,'Deck',deck)
    #df.insert(3,'Num',num)
    df.insert(3,'Side',side)
    return(df)

In [57]:
def drop_redundant_cols(df):
    df = df.drop(['PassengerId','Name'], axis = 1)
    return(df)

In [58]:
def deal_with_NaNs(df):
    df['HomePlanet'] = df['HomePlanet'].fillna('Earth')
    df['CryoSleep'] = df['CryoSleep'].fillna(False)
    df['Destination'] = df['Destination'].fillna('TRAPPIST-1e')
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['VIP'] = df['VIP'].fillna(False)
    df['RoomService'] = df['RoomService'].fillna(df['RoomService'].mean())
    df['FoodCourt'] = df['FoodCourt'].fillna(df['FoodCourt'].mean())
    df['ShoppingMall'] = df['ShoppingMall'].fillna(df['ShoppingMall'].mean())
    df['Spa'] = df['Spa'].fillna(df['Spa'].mean())
    df['VRDeck'] = df['VRDeck'].fillna(df['VRDeck'].mean())
    df['Cabin'] = df['Cabin'].fillna(method = 'ffill')
    

    return(df)

In [59]:
def OneHot(df):
    df = pd.get_dummies(df)
    return(df)

In [60]:
def label_feat_split(df):
    labels = df['Transported']
    features = df.drop(['Transported'],axis=1)
    return features, labels

In [61]:
def Get_Final_data(df):
    df = deal_with_NaNs(df)
    df = drop_redundant_cols(df)
    df = data_to_int(df)
    df= OneHot(df)
    return(df)


In [62]:
def Output_File(ids,Answers):
    dic = {"PassengerId":ids , "Transported":Answers}
    final = pd.DataFrame(dic)
    final['Transported'] = final['Transported'].astype(bool)
    print(final)
    final.to_csv("Answers.csv", index=False)

In [63]:
def grid_search(clf,X_train,Y_train):
    scorer = make_scorer(accuracy_score)
    parameters = {"n_estimators":[30,40,50,60,70],"learning_rate":[0.01,0.05,0.1,0.5,1]}
    grid_obj = GridSearchCV(clf,parameters,scoring=scorer)
    grid_fit = grid_obj.fit(X_train,Y_train)
    bestclf = grid_fit.best_estimator_
    return(bestclf)

In [64]:
train_data_final = Get_Final_data(train_data)
features, labels = label_feat_split(train_data_final)
X_train , X_test, Y_train , Y_test = train_test_split(features, labels, test_size=0.25, random_state=45)
print("Training data is splitted")

len deck = 8693
len side = 8693
Training data is splitted


In [65]:
'''clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test ,Y_train,Y_test)
models'''

'clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)\nmodels, predictions = clf.fit(X_train, X_test ,Y_train,Y_test)\nmodels'

In [66]:
#print(train_data_final.head())
clf = LGBMClassifier()
clf.fit(X_train,Y_train)
Y_predict = clf.predict(X_test)

print("Accurancy Score = {}".format(accuracy_score(Y_test,Y_predict)))

'''best_clf = grid_search(clf,X_train,Y_train)
Y_predict = best_clf.predict(X_test)

print("Accurancy Score = {}".format(accuracy_score(Y_test,Y_predict)))'''

Accurancy Score = 0.8114075436982521


'best_clf = grid_search(clf,X_train,Y_train)\nY_predict = best_clf.predict(X_test)\n\nprint("Accurancy Score = {}".format(accuracy_score(Y_test,Y_predict)))'

In [67]:
ids = test_data['PassengerId']
test_feats_final = Get_Final_data(test_data)
Answers = clf.predict(test_feats_final)
print(Answers)
Output_File(ids,Answers)

len deck = 4277
len side = 4277
[1 0 1 ..., 1 1 1]
     PassengerId  Transported
0        0013_01         True
1        0018_01        False
2        0019_01         True
3        0021_01         True
4        0023_01         True
5        0027_01        False
6        0029_01         True
7        0032_01         True
8        0032_02         True
9        0033_01         True
10       0037_01        False
11       0040_01        False
12       0040_02         True
13       0042_01         True
14       0046_01        False
15       0046_02        False
16       0046_03        False
17       0047_01         True
18       0047_02         True
19       0047_03        False
20       0048_01         True
21       0049_01        False
22       0054_01         True
23       0054_02         True
24       0054_03         True
25       0055_01        False
26       0057_01         True
27       0059_01         True
28       0060_01         True
29       0063_01         True
...          ...   